# Generation of BoM Data for Supply Chain Simulation

In [82]:
SAVE_RECORDS=True

In [83]:
import sys
sys.path.append("..")

In [84]:
import random
import pandas as pd
import json

## Mock Supply Chain

In [85]:
from core.supply_chain import SupplyChain, LeafNode, CombinerNode, SinkNode

def create_supply_chain():
    supply_chain = SupplyChain()

    # Create leaf nodes with random cost types
    leaf1 = LeafNode(200, "raw_material_1", [(0, 10), (5, 15), (10, 20)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    leaf2 = LeafNode(250, "raw_material_2", [(0, 15), (5, 20), (10, 25)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    leaf3 = LeafNode(220, "raw_material_3", [(0, 12), (5, 18), (10, 22)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(leaf1)
    supply_chain.add_node(leaf2)
    supply_chain.add_node(leaf3)

    # Create intermediate nodes with random cost types
    intermediate_a1 = CombinerNode(300, "intermediate_a1", [0.8, 1.0], [1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    intermediate_a2 = CombinerNode(280, "intermediate_a2", [0.9, 1.1], [1.1, 1.3], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(intermediate_a1)
    supply_chain.add_node(intermediate_a2)

    intermediate_b1 = CombinerNode(300, "intermediate_b1", [0.8, 1.0], [1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    intermediate_b2 = CombinerNode(280, "intermediate_b2", [0.9, 1.1], [1.1, 1.3], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(intermediate_b1)
    supply_chain.add_node(intermediate_b2)

    # Create final combiner node with random cost type
    final_combiner = CombinerNode(350, "final_product", [0.7, 0.8, 0.9], [1, 1.1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(final_combiner)

    # Create sink node with random cost type
    sink = SinkNode(consumption_rate=250, cost_type=random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(sink)

    # Add edges with cost ranges
    supply_chain.add_edge(leaf1, intermediate_a1, 10, 30, 5, 15)
    supply_chain.add_edge(leaf2, intermediate_a1, 15, 35, 8, 20)
    supply_chain.add_edge(leaf2, intermediate_a2, 12, 30, 6, 18)
    supply_chain.add_edge(leaf3, intermediate_a2, 14, 32, 7, 19)
    supply_chain.add_edge(intermediate_a1, intermediate_b1, 20, 40, 10, 25)
    supply_chain.add_edge(intermediate_a2, intermediate_b1, 22, 42, 11, 27)
    supply_chain.add_edge(intermediate_b1, final_combiner, 20, 40, 10, 25)
    supply_chain.add_edge(intermediate_b2, final_combiner, 22, 42, 11, 27)
    supply_chain.add_edge(leaf3, final_combiner, 18, 35, 9, 23)
    supply_chain.add_edge(final_combiner, sink, 25, 50, 13, 30)

    return supply_chain

In [86]:
# Simulation
time = 0
supply_chain = create_supply_chain()

for _ in range(200):  # Simulate for 200 time steps
    supply_chain.update()
    time += 1

    # Print current state
    print(f"Time: {time}")
    for node in supply_chain.nodes:
        print(f"Node ({node.node_class}): Inventory = {node.inventory}, Last Production = {node.last_production}, Cost Type = {node.cost_type}")
        for edge, target_node in node.outgoing_edges:
            print(f"  Edge to {target_node.node_class}: Quantity = {edge.quantity}, Cost = {edge.current_cost:.2f}")
    print("---")

Time: 1
Node (raw_material_1): Inventory = 0, Last Production = 10, Cost Type = fixed
  Edge to intermediate_a1: Quantity = 30, Cost = 10.00
Node (raw_material_2): Inventory = 1, Last Production = 15, Cost Type = negative_dynamic
  Edge to intermediate_a1: Quantity = 35, Cost = 19.66
  Edge to intermediate_a2: Quantity = 30, Cost = 17.95
Node (raw_material_3): Inventory = 1, Last Production = 12, Cost Type = positive_dynamic
  Edge to intermediate_a2: Quantity = 32, Cost = 7.38
  Edge to final_product: Quantity = 35, Cost = 9.06
Node (intermediate_a1): Inventory = 0, Last Production = 95, Cost Type = fixed
  Edge to intermediate_b1: Quantity = 40, Cost = 17.50
Node (intermediate_a2): Inventory = 0, Last Production = 137, Cost Type = positive_dynamic
  Edge to intermediate_b1: Quantity = 42, Cost = 11.00
Node (intermediate_b1): Inventory = 0, Last Production = 120, Cost Type = fixed
  Edge to final_product: Quantity = 40, Cost = 17.50
Node (intermediate_b2): Inventory = 0, Last Producti

## Recording of the simulation

In [87]:
def simulate_and_collect_data(supply_chain, n_cycles):
    # Initialize DataFrames
    metadata = []
    node_data = []
    edge_data = []
    
    node_id = 0

    # Create metadata and assign unique IDs
    for node in supply_chain.nodes:
        metadata.append({
            'node_id': str(node_id), 
            'node_class': node.node_class,
            'max_inventory': node.max_inventory,
            'cost_type': node.cost_type
        })
        node.id = node_id  # Assign ID to node object for reference
        node_id += 1

    edge_id = 0
    # Create edge metadata
    for edge in supply_chain.edges:
        source_node = None
        target_node = None
        
        for node in supply_chain.nodes:
            if edge in [e for e, _ in node.outgoing_edges]:
                source_node = node
            if edge in [e for e, _ in node.incoming_edges]:
                target_node = node
            if source_node and target_node:
                break
        
        if not source_node or not target_node:
            print(f"Warning: Edge {edge_id} is not properly connected.")
            print(f"Source node: {source_node.node_class if source_node else 'None'}")
            print(f"Target node: {target_node.node_class if target_node else 'None'}")
            continue
        
        metadata.append({
            'edge_id': str(edge_id),
            'source_node_id': str(source_node.id),
            'target_node_id': str(target_node.id),
            'unit_price': edge.unit_price,
            'min_cost': edge.min_cost,
            'max_cost': edge.max_cost
        })
        edge.id = edge_id  # Assign ID to edge object for reference
        edge_id += 1

    # Simulate for n cycles
    for cycle in range(n_cycles):
        supply_chain.update()

        # Collect node data
        for node in supply_chain.nodes:
            node_data.append({
                'cycle': cycle,
                'node_id': str(node.id),
                'inventory': node.inventory,
                'last_production': node.last_production
            })

        # Collect edge data
        for edge in supply_chain.edges:
            if hasattr(edge, 'id'):  # Only collect data for edges that were properly connected
                edge_data.append({
                    'cycle': cycle,
                    'edge_id': str(edge.id),
                    'quantity': edge.quantity,
                    'current_cost': edge.current_cost
                })

    # Create DataFrames
    metadata_df = pd.DataFrame(metadata)
    node_data_df = pd.DataFrame(node_data)
    edge_data_df = pd.DataFrame(edge_data)

    return metadata_df, node_data_df, edge_data_df

In [88]:
supply_chain = create_supply_chain()
metadata_df, node_data_df, edge_data_df = simulate_and_collect_data(supply_chain, n_cycles=20)

In [89]:
metadata_df

,node_id,node_class,max_inventory,cost_type,edge_id,source_node_id,target_node_id,unit_price,min_cost,max_cost
0,0,raw_material_1,200.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
1,1,raw_material_2,250.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
2,2,raw_material_3,220.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
3,3,intermediate_a1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
4,4,intermediate_a2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
5,5,intermediate_b1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
6,6,intermediate_b2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
7,7,final_product,350.0,positive_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
8,8,sink,inf,positive_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,0,0,3,10.0,5.0,15.0


In [90]:
node_data_df

,cycle,node_id,inventory,last_production
0,0,0,0,10
1,0,1,1,15
2,0,2,1,12
3,0,3,0,95
4,0,4,0,137
...,...,...,...,...
175,19,4,0,15
176,19,5,0,92
177,19,6,0,1
178,19,7,0,112


In [91]:
edge_data_df

,cycle,edge_id,quantity,current_cost
0,0,0,30,15.0
1,0,1,35,14.0
2,0,2,30,12.0
3,0,3,32,13.0
4,0,4,40,17.5
...,...,...,...,...
195,19,5,31,27.0
196,19,6,147,17.5
197,19,7,1,27.0
198,19,8,7,16.0


## Conversion between JSON and DataFrames

In [92]:
from typing import Dict, List

def dataframes_to_json(metadata: pd.DataFrame, node: pd.DataFrame, edge: pd.DataFrame) -> str:
    bom = {
        "metadata": metadata.to_dict(orient='records'),
        "nodes": {},
        "edges": {}
    }

    # Process node data
    for _, row in node.iterrows():
        node_id = row['node_id']
        cycle = row['cycle']
        if node_id not in bom["nodes"]:
            bom["nodes"][node_id] = {}
        bom["nodes"][node_id][cycle] = row.to_dict()

    # Process edge data
    for _, row in edge.iterrows():
        edge_id = row['edge_id']
        cycle = row['cycle']
        if edge_id not in bom["edges"]:
            bom["edges"][edge_id] = {}
        bom["edges"][edge_id][cycle] = row.to_dict()

    return json.dumps(bom, indent=2)

def json_to_dataframes(json_data: str) -> Dict[str, pd.DataFrame]:
    bom = json.loads(json_data)

    # Create empty lists to store the data
    node_data = []
    edge_data = []

    # Process nodes
    for node_id, cycles in bom["nodes"].items():
        for cycle, node_info in cycles.items():
            node_data.append(node_info)

    # Process edges
    for edge_id, cycles in bom["edges"].items():
        for cycle, edge_info in cycles.items():
            edge_data.append(edge_info)

    # Create DataFrames
    metadata_df = pd.DataFrame(bom["metadata"])
    node_df = pd.DataFrame(node_data)
    edge_df = pd.DataFrame(edge_data)

    return {
        "metadata": metadata_df,
        "node": node_df,
        "edge": edge_df
    }

In [93]:
# Convert dataframes to JSON
json_string = dataframes_to_json(metadata_df, node_data_df, edge_data_df)

In [94]:
json_data = json.loads(json_string)
json_data

{'metadata': [{'node_id': '0',
   'node_class': 'raw_material_1',
   'max_inventory': 200.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '1',
   'node_class': 'raw_material_2',
   'max_inventory': 250.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '2',
   'node_class': 'raw_material_3',
   'max_inventory': 220.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '3',
   'node_class': 'intermediate_a1',
   'max_inventory': 300.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '4

In [95]:
if SAVE_RECORDS:
    with open('../data/json/true.json', 'w') as f:
        json.dump(json_data, f, ensure_ascii=False)

In [96]:
# Convert JSON back to dataframes
reconstructed_dfs = json_to_dataframes(json_string)

# Access the reconstructed dataframes
reconstructed_metadata = reconstructed_dfs['metadata']
reconstructed_node = reconstructed_dfs['node']
reconstructed_edge = reconstructed_dfs['edge']

In [97]:
if SAVE_RECORDS:
    reconstructed_metadata.to_csv('../data/csv/metadata.csv', index=False)
    reconstructed_node.to_csv('../data/csv/node.csv', index=False)
    reconstructed_edge.to_csv('../data/csv/edge.csv', index=False)

In [98]:
reconstructed_metadata

,node_id,node_class,max_inventory,cost_type,edge_id,source_node_id,target_node_id,unit_price,min_cost,max_cost
0,0,raw_material_1,200.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
1,1,raw_material_2,250.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
2,2,raw_material_3,220.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
3,3,intermediate_a1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
4,4,intermediate_a2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
5,5,intermediate_b1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
6,6,intermediate_b2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
7,7,final_product,350.0,positive_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
8,8,sink,inf,positive_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,0,0,3,10.0,5.0,15.0


In [99]:
reconstructed_node

,cycle,node_id,inventory,last_production
0,0,0,0,10
1,1,0,0,10
2,2,0,0,10
3,3,0,0,10
4,4,0,0,10
...,...,...,...,...
175,15,8,538,250
176,16,8,555,250
177,17,8,594,250
178,18,8,611,250


In [100]:
reconstructed_edge 

,cycle,edge_id,quantity,current_cost
0,0,0,30,15.0
1,1,0,10,15.0
2,2,0,10,15.0
3,3,0,10,15.0
4,4,0,10,15.0
...,...,...,...,...
195,15,9,289,13.0
196,16,9,267,13.0
197,17,9,289,13.0
198,18,9,267,13.0


## Aggredation of Cycles and Conversion between JSON and Dataframes

In [101]:
from typing import Dict

def aggregated_dataframes_to_json(metadata: pd.DataFrame, node: pd.DataFrame, edge: pd.DataFrame, max_aggregation_cycles: int = 3) -> str:
    bom = {
        "metadata": metadata.to_dict(orient='records'),
        "nodes": {},
        "edges": {}
    }

    # Initialize variables to track the last cycle
    last_node_cycle = 0
    last_edge_cycle = 0
    
    num_cycles_to_aggregate = random.randint(1, max_aggregation_cycles)

    # Process node data with aggregation
    for _, row in node.iterrows():
        node_id = row['node_id']
        cycle = row['cycle']
        # Check if we can aggregate
        if node_id not in bom["nodes"]:
            bom["nodes"][node_id] = {}
        
        # Aggregate nodes
        for _ in range(num_cycles_to_aggregate):
            last_node_cycle += 1
            # Create a new entry for the aggregated node
            aggregated_node = row.copy()
            aggregated_node['cycle'] = last_node_cycle
            bom["nodes"][node_id][last_node_cycle] = aggregated_node.to_dict()

    # Process edge data with aggregation
    for _, row in edge.iterrows():
        edge_id = row['edge_id']
        cycle = row['cycle']
        # Check if we can aggregate
        if edge_id not in bom["edges"]:
            bom["edges"][edge_id] = {}
        
        # Aggregate edges
        for _ in range(num_cycles_to_aggregate):
            last_edge_cycle += 1
            # Create a new entry for the aggregated edge
            aggregated_edge = row.copy()
            aggregated_edge['cycle'] = last_edge_cycle
            bom["edges"][edge_id][last_edge_cycle] = aggregated_edge.to_dict()

    return json.dumps(bom, indent=2)

In [102]:
aggregated_json = aggregated_dataframes_to_json(metadata_df, node_data_df, edge_data_df)
aggregated_json_dict = json.loads(aggregated_json)

aggregated_json_dict

{'metadata': [{'node_id': '0',
   'node_class': 'raw_material_1',
   'max_inventory': 200.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '1',
   'node_class': 'raw_material_2',
   'max_inventory': 250.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '2',
   'node_class': 'raw_material_3',
   'max_inventory': 220.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '3',
   'node_class': 'intermediate_a1',
   'max_inventory': 300.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': '4

In [103]:
if SAVE_RECORDS:
    import json
    with open('../data/json/aggregated.json', 'w') as f:
        json.dump(aggregated_json_dict, f, ensure_ascii=False)

In [104]:
aggregated_dataframe = json_to_dataframes(aggregated_json)

aggregated_metadata = aggregated_dataframe["metadata"]
aggregated_nodes = aggregated_dataframe["node"]
aggregated_edges = aggregated_dataframe["edge"]

In [105]:
if SAVE_RECORDS:
    aggregated_metadata.to_csv("../data/csv/aggregated_metadata.csv", index=False)
    aggregated_nodes.to_csv("../data/csv/aggregated_node_data.csv", index=False)
    aggregated_edges.to_csv("../data/csv/aggregated_edge_data.csv", index=False)

In [106]:
aggregated_metadata

,node_id,node_class,max_inventory,cost_type,edge_id,source_node_id,target_node_id,unit_price,min_cost,max_cost
0,0,raw_material_1,200.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
1,1,raw_material_2,250.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
2,2,raw_material_3,220.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
3,3,intermediate_a1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
4,4,intermediate_a2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
5,5,intermediate_b1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
6,6,intermediate_b2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
7,7,final_product,350.0,positive_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
8,8,sink,inf,positive_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,0,0,3,10.0,5.0,15.0


In [107]:
aggregated_nodes

,cycle,node_id,inventory,last_production
0,1,0,0,10
1,2,0,0,10
2,3,0,0,10
3,28,0,0,10
4,29,0,0,10
...,...,...,...,...
535,512,8,611,250
536,513,8,611,250
537,538,8,650,250
538,539,8,650,250


In [108]:
aggregated_edges

,cycle,edge_id,quantity,current_cost
0,1,0,30,15.0
1,2,0,30,15.0
2,3,0,30,15.0
3,31,0,10,15.0
4,32,0,10,15.0
...,...,...,...,...
595,569,9,267,13.0
596,570,9,267,13.0
597,598,9,289,13.0
598,599,9,289,13.0
